### 의류 종류 인식
의류의 분류별로 인식하게끔 하는 과정을 진행해보자.

### 데이터 수집
크롤링으로 하이버에서 종류별로 이미지를 가져온다.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys


# 옵션 정의
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-gpu')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")

In [2]:
# 스크롤 다운 함수 정의

def scroll_down(body):
    for i in range(1, 10):
        body.send_keys(Keys.PAGE_DOWN)
        body.send_keys(Keys.PAGE_DOWN)
        body.send_keys(Keys.PAGE_DOWN)
        body.send_keys(Keys.PAGE_DOWN)

### Image Url 수집

In [4]:
import time
from selenium.webdriver.common.by import By

driver = webdriver.Chrome("../114/chromedriver.exe", options = options)

label_dict = {1 : "short", 2 : "long", 3 : "man2man", 4 : "hood-zip_up", 5 : "knitwear"}

src_list = []
for i, value in label_dict.items():
    url = f"https://www.hiver.co.kr/categories/31{i}"
    driver.get(url)
    driver.implicitly_wait(5)
    body = driver.find_element(By.CSS_SELECTOR, "body")
    
    for i in range(3):
        scroll_down(body)
        
        show_more = driver.find_element(By.CSS_SELECTOR, "#container > div > div > section.section.css-hmo7v7 > div.css-ducvkm > button")
        show_more.click()
        time.sleep(3)
        
        scroll_down(body)
    
    imgs = driver.find_elements(By.CLASS_NAME, "css-eivff4")
    img_src = []
    for img in imgs:
        src = img.get_attribute("src")
        img_src.append(src)
        if img_src == 40:
            break
        
    src_list.append(img_src[:100])

driver.close()

### 수집된 URL로부터 이미지 다운로드

In [5]:
import requests

headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"}

label_dict = {1 : "short", 2 : "long", 3 : "man2man", 4 : "hoodie", 5 : "knitwear"}

i = 0
for idx, srcs in enumerate(src_list):
    for src in srcs:
        req = requests.get(src, headers = headers)
        with open(f"./images/{i}_{label_dict[idx+1]}.jpeg", "wb") as file:
            file.write(req.content)
        i += 1

### 이미지 라벨링1 (1, 2중 택1)
labelImg를 활용한 라벨링

#### Download
1. https://github.com/tzutalin/labelImg 우측 하단의 릴리즈 클릭
2. 버전에 맞게 다운로드 후 cmd 내부에서 해당 폴더의 위치로 이동
3. 아래 순서대로 실행
```
>>> pip install PyQt5
>>> pip install lxml
>>> pyrcc5 -o libs/resources.py resources.qrc 
>>> pip3 install labelImg 
>>> python labelimg.py
```

> pyrcc5 -o libs/resources.py resources.qrc 의 경우 pyqt에 대한 환경변수 설정이 되어 있어야 함. <p>
> 환경변수 편집 ➡️ 시스템 변수 ➡️ 새로 만들기 ➡️ 변수이름 : pyrcc5, 변수값 : PyQt5가 설치된 경로

#### Usage
0. 설치된 폴더들을 살펴보면 data란 폴더가 있는데, 그 안에 있는 predefined_classes.txt파일을 눌러서 본인이 분류하고자 하는 클래스 이름으로 변경해야한다.
1. `python labelimg.py`이 실행되면 GUI 창 하나가 켜진다. 좌측 하단의 Yolo 버튼을 세번 눌러서 다시 Yolo로 맞춘다.
2. 편집할 이미지가 담긴 디렉토리를 불러온다.
3. w를 누르면 커서가 바뀌면서 어노테이션 시작. Save버튼 누르는거 잊지 않기...

In [ ]:
%cd C:\Users\OWNER\Desktop\labeling\labelImg-master
!python labelimg.py

In [ ]:
%cd D:\Intel\YOLO

### 이미지 라벨링2
roboflow를 활용한 라벨링

1. Roboflow 회원가입(WorkSpace생성)
2. 프로젝트 생성
    + 2-1. Project Type : Object detection
    + 2-2. What Are You Detecting? : 분류하고자 하는 것들의 대분류(여기서는 옷을 의미하는 cloth)
    + 2-3. Project Name : 프로젝트가 생성될 이름(저는 cloth detection으로 생성했습니다.)
    + 2-4. License : CC BY 4.0
3. 이미지 폴더 업로드 후 우측 상단 `Save and Continue` 클릭 이후 오른쪽에서 탭 하나가 튀어나올텐데 하단에 `Assing Image`클릭
4. 우측 상단의 `Start Annotating` 누르며 어노테이션 시작(첫 이미지 어노테이션은 바로 되는데 두번째부터는 ctrl을 누르고 진행해야함)
5. 어노테이션 완료 후 Generate탭 클릭
6. 차례대로 진행
7. 우측 상단의 Export Dataset 클릭(Format 형태 : YoloV5 Pytorch, 다운로드 형태 선택)
8. 다운된 데이터를 기반으로 훈련

### clone yolov5

In [ ]:
!git clone https://github.com/ultralytics/yolov5

### Yaml 파일 수정

In [2]:
import yaml

with open('./yolov5/data/coco.yaml', "r") as f:
    data = yaml.safe_load(f)
    
data["path"] = r"D:\Intel\YOLO\Roboflow"
data["train"] = "train"
data["test"] = "test"
data["val"] = "valid"

data["nc"] = 5
data["names"] = ['short', 'long', 'man2man', 'hoodie', 'knitwear']

with open("./yolov5/data/coco.yaml", "w") as file:
    yaml.dump(data, file)

In [3]:
%pwd

'd:\\Intel\\YOLO'

### 훈련
clone한 파일들 중 `train.py`를 이용해서 훈련을 진행한다. argparse로 작성되어 여러 옵션들을 함께 지정해주어야 한다.
- --batch : batch_size
- --epochs : 훈련 횟수
- --data : data정보를 담고있는 yaml 파일
- --weights : pretrained 된 가중치를 적용할 때 사용. 지정하지 않으면 랜덤한 가중치와 편향값들로 학습 진행
- --cfg : 모델을 담고있는 yaml파일. s ➡️ n ➡️ m ➡️ l ➡️ x 순으로 복잡도가 다르다. 

In [ ]:
!python ./YOLO/yolov5/train.py --batch 4 --epochs 200 --data ./YOLO/yolov5/data/coco.yaml  --weights ./YOLO/yolov5/yolov5n.pt --cfg ./YOLO/yolov5/models/yolov5m.yaml

### 검출
위에서 훈련한 나만의 모델을 기반으로 새로운 이미지들에 대한 객체 탐지 결과를 `detect.py`파일로 받아본다. 마찬가지로 argparse로 작성되어있다.
- --source : 탐지할 이미지 경로
- --weights : 나의 학습 모델의 가중치 경로
- --conf : 객체 탐지 신뢰도. 해당 신뢰도 이상의 객체들만 탐지한다. 기본값은 0.4이며 높을 수록 탐지되는 객체의 수도 적어진다.
- --name : 탐지된 파일들을 저장할 경로

In [ ]:
!python yolov5/detect.py --source "./YOLO/Roboflow/test/images" --weights "./YOLO/yolov5/runs/train/exp4/weights/best.pt" --conf 0.3 --name "./YOLO/yolov5m_clothes_detections"

### 결과
학습된 모델의 상태가 좋지 않다. 원인을 따져보자면 아래와 같을 것이다.
- 클래스 간의 상관관계가 너무 높음
    + 현재 구성한 클래스는 [반팔, 긴팔, 맨투맨, 후드, 니트]로, 니트의 경우 반팔 니트, 긴팔 니트가 존재하는 등의 상관관계가 높은 것이 아닌가 하는 생각이 든다
- 노이즈
    + Roboflow에서 제공하는 이미지 증강이 오히려 역효과를 낸 건 아닐까 하는 마음.. 
    + https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=4u_olion&logNo=221437862590 참고